In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder\
        .appName('streaming-sink-source')\
        .enableHiveSupport()\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/01 22:47:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql(""" create table if not exists flights_tble 
using csv options(path '/Users/xwyang/Desktop/data/flight_delays.csv')     """)

24/07/01 22:47:38 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/07/01 22:47:38 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/07/01 22:47:42 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/07/01 22:47:42 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore xwyang@192.168.12.190


DataFrame[]

In [4]:
spark.sql('show tables').show()

24/07/01 22:47:43 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default|flights_tble|      false|
|  default|    fly_tble|      false|
|  default|    flys_tbl|      false|
|  default|    new_tble|      false|
+---------+------------+-----------+



In [6]:
spark.sql('drop table flys_tbl')

DataFrame[]

In [7]:
spark.sql(""" create or replace temp view  flights_view
using csv options(path '/Users/xwyang/Desktop/data/flight_delays.csv')     """)

DataFrame[]

In [8]:
df = spark.sql("select * from flights_view")

In [9]:
df.show(5)

+--------+-----+--------+------+-----------+
|     _c0|  _c1|     _c2|   _c3|        _c4|
+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 5 rows



In [10]:
data = spark.read.csv('/Users/xwyang/Desktop/data/flight_delays.csv',header=True,inferSchema=True)

In [11]:
data.show(5,False)

+-------+-----+--------+------+-----------+
|date   |delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|6    |602     |ABE   |ATL        |
|1020600|-8   |369     |ABE   |DTW        |
|1021245|-2   |602     |ABE   |ATL        |
|1020605|-4   |602     |ABE   |ATL        |
|1031245|-4   |602     |ABE   |ATL        |
+-------+-----+--------+------+-----------+
only showing top 5 rows



In [13]:
data.write.format('parquet').saveAsTable('fly_tble',mode='overwrite')

24/07/01 22:49:46 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/07/01 22:49:46 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/07/01 22:49:46 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/07/01 22:49:46 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [14]:
spark.table('fly_tble').show(5)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1010630|  -10|     928|   RSW|        EWR|
|1021029|   87|     974|   RSW|        ORD|
|1021346|    0|     928|   RSW|        EWR|
|1021044|   18|     928|   RSW|        EWR|
|1021730|   29|     748|   RSW|        IAH|
+-------+-----+--------+------+-----------+
only showing top 5 rows



In [15]:
data.createOrReplaceTempView('delays_tble')

In [16]:
spark.sql('show tables').show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default|flights_tble|      false|
|  default|    fly_tble|      false|
|  default|    new_tble|      false|
|         | delays_tble|       true|
|         |flights_view|       true|
+---------+------------+-----------+



In [17]:
import tempfile
import time
with tempfile.TemporaryDirectory() as d:
    # Write a temporary text file to read it.
    spark.createDataFrame([(1, "2"),]).write.mode("overwrite").format("csv").save(d)

    # Start a streaming query to read the CSV file.
    q = spark.readStream.schema(
        "col0 INT, col1 STRING"
    ).format("csv").load(d).writeStream.format("console").start()
    time.sleep(3)
    q.stop()

24/07/01 22:50:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/xs/5xq0dlqn045flbfc8gv1qn140000gn/T/temporary-4120b1e0-0fd4-4662-919c-fa9fa8c953f9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/01 22:50:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----+
|col0|col1|
+----+----+
|   1|   2|
+----+----+



In [18]:
import tempfile
import time
with tempfile.TemporaryDirectory() as d:
    # Write a temporary text file to read it.
    spark.createDataFrame(
        [("hello",), ("this",)]).write.mode("overwrite").format("text").save(d)

    # Start a streaming query to read the text file.
    q = spark.readStream.format("text").load(d).writeStream.format("console").start()
    time.sleep(3)
    q.stop()

24/07/01 22:50:22 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/xs/5xq0dlqn045flbfc8gv1qn140000gn/T/temporary-15a9322b-a61b-484c-8a2a-fdd2be9d4fd7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/01 22:50:22 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
|hello|
| this|
+-----+



In [20]:
spark.sql("""create table if not exists new_tble as select * from fly_tble   """)

24/07/01 22:50:59 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


DataFrame[]

In [21]:
spark.table('new_tble').show(5,False)

+-------+-----+--------+------+-----------+
|date   |delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|3300630|-4   |473     |LGA   |CLT        |
|3301200|-2   |160     |LGA   |BOS        |
|3301400|-5   |160     |LGA   |BOS        |
|3301600|46   |160     |LGA   |BOS        |
|3301800|0    |160     |LGA   |BOS        |
+-------+-----+--------+------+-----------+
only showing top 5 rows



In [22]:
da = [("Alice", 25), ("Bob", 30), ("Cathy", 28)]
dff = spark.createDataFrame(da, ["Name", "Age"])
dff.show()


+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
|Cathy| 28|
+-----+---+



Certainly! Here's the explanation and complete example in English:

### Overview of Steps

1. **Create SparkSession**: Initialize a SparkSession object.
2. **Define Schema**: Define the schema for the CSV files.
3. **Specify Directory Path**: Ensure the directory path is valid and contains CSV files.
4. **Read Streaming CSV Files**: Use Structured Streaming to read CSV files from the directory.
5. **Write Data to Console or Other Sink**: Write the streamed data to the console or another supported output sink.

### Example Code

Here is a complete example showing how to stream read CSV files from a directory and output the data to the console:

```python
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Create SparkSession
spark = SparkSession.builder \
    .appName("Streaming CSV Files") \
    .getOrCreate()

# Define Schema
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True),
    StructField("Country", StringType(), True)
])

# Specify CSV directory path
csv_dir = "/path/to/your/csv_directory"

# Check if directory exists
if not os.path.exists(csv_dir):
    raise FileNotFoundError(f"The directory {csv_dir} does not exist.")

# Check if there are CSV files in the directory
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in the directory {csv_dir}.")

print(f"CSV files found: {csv_files}")

# Read streaming CSV files
df = spark.readStream \
    .schema(schema) \
    .csv(csv_dir)

# Write data to console
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the streaming query to finish
query.awaitTermination()
```

### Detailed Explanation

1. **Create SparkSession**:
   ```python
   spark = SparkSession.builder \
       .appName("Streaming CSV Files") \
       .getOrCreate()
   ```
   This initializes a SparkSession, which is the entry point for using Spark functionality.

2. **Define Schema**:
   ```python
   schema = StructType([
       StructField("Name", StringType(), True),
       StructField("Age", IntegerType(), True),
       StructField("Country", StringType(), True)
   ])
   ```
   Defines the schema of the CSV files, ensuring all files have the same structure.

3. **Specify Directory Path and Check**:
   ```python
   csv_dir = "/path/to/your/csv_directory"

   # Check if directory exists
   if not os.path.exists(csv_dir):
       raise FileNotFoundError(f"The directory {csv_dir} does not exist.")

   # Check if there are CSV files in the directory
   csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
   if not csv_files:
       raise FileNotFoundError(f"No CSV files found in the directory {csv_dir}.")

   print(f"CSV files found: {csv_files}")
   ```
   Ensures the specified directory exists and contains CSV files. If not, raises an appropriate error.

4. **Read Streaming CSV Files**:
   ```python
   df = spark.readStream \
       .schema(schema) \
       .csv(csv_dir)
   ```
   Uses the `readStream` method to read CSV files from the specified directory.

5. **Write Data to Console**:
   ```python
   query = df.writeStream \
       .outputMode("append") \
       .format("console") \
       .start()

   query.awaitTermination()
   ```
   Writes the streamed data to the console and starts the streaming query, then waits for it to terminate.

By following these steps, you can use PySpark to stream read multiple CSV files from a directory, given that they all have the same schema, and process the data accordingly.

In [37]:
import os
os.listdir('/Users/xwyang/Desktop/csv_dir')

['1.csv', '3.csv', '2.csv']

In [24]:
data.show(2)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
+-------+-----+--------+------+-----------+
only showing top 2 rows



In [25]:
data.limit(10).show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+



In [28]:
data.select('distance','delay','origin').distinct().limit(10).show()

+--------+-----+------+
|distance|delay|origin|
+--------+-----+------+
|     369|   -9|   ABE|
|    1103|  489|   ABQ|
|     124|  125|   ALB|
|     932|  179|   ALB|
|    1047|   51|   ALB|
|     271|   30|   AMA|
|     496|    5|   ANC|
|    1340|   46|   ANC|
|    1259|  -13|   ANC|
|     227|    7|   ANC|
+--------+-----+------+



In [35]:
data.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [31]:
import os
dataframes=[]
file_dir='/Users/xwyang/Desktop/csv_dir'
files = [file for file in os.listdir(file_dir)]
for file in files:
    df = pd.read_csv(os.path.join(file_dir,file))
    dataframes.append(df)
    



In [38]:
for dd in dataframes:
    print(dd.columns,len(dd))

Index(['date', 'delay', 'distance', 'origin', 'destination'], dtype='object') 1391578
Index(['date', 'delay', 'distance', 'origin', 'destination'], dtype='object') 1391578
Index(['date', 'delay', 'distance', 'origin', 'destination'], dtype='object') 1391578


In [36]:
schema_file = "date int,delay int,distnace int,origin string,destination string    "
stream_df = spark.readStream.format('csv')\
                 .option('header','true')\
                 .schema(schema_file)\
                 .load(file_dir)

In [ ]:
check_dir = '/Users/xwyang/Desktop/check_dir'
query=stream_df.writeStream.format('csv')\
               .outputMode('append')\
               .option('path',file_dir)\
               .option('checkpointLocation',check_dir)\
               .start()
query.awaitTermination()
      
       

24/07/01 23:31:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
